# INSTALAR REPORTLAB

In [453]:
%pip install reportlab

Note: you may need to restart the kernel to use updated packages.


In [454]:
%pip install untangle

Note: you may need to restart the kernel to use updated packages.


# IMPORTAR LIBRERIAS

In [455]:
import os
import untangle
import pandas as pd

## Lectura de XML

In [456]:
def get_info_from_xml(xml_path):
    doc = untangle.parse(xml_path)
    
    # Movemos el root a "Clashtest"
    clashtest = doc.exchange.batchtest.clashtests.clashtest
    
    # Extraemos la metadata de clashtest
    nombre_archivo = clashtest["name"]
    tolerance = clashtest["tolerance"]
    
    # Buscamos el summary de clashtest y obtenemos sus datos
    summary = clashtest.summary
    total = summary["total"]
    activas = summary["active"]
    resueltas = summary["resolved"]

    clash_results = clashtest.clashresults
    # Creamos un dataframe vacio donde se almacenaran las columnas de interes
    df = pd.DataFrame(columns=['Nombre', 'Imagen', 'Distancia', 'Estado', 'Grid', 'Puntos', 'Fecha', 'Objetos'])
    
    
    i = 0
    for clashresult in clash_results.clashresult:
        # Datos del campo "clashresult"
        nombre_conflicto = clashresult['name']
        imagen_filename = clashresult['href']
        
        # Eliminamos el "nombre_archivo" de este path, ya que para algunos casos está malo 
        # (carpeta 06) difiere el nombre_archivo del xml de la ruta real de la imagen
        imagen_filename = imagen_filename.split('\\')[-1]
        
        distancia = clashresult['distance']
        
        result_status = clashresult.resultstatus.cdata
        gridlocation = clashresult.gridlocation.cdata
        
        # Datos del campo "clashpoint" para obtener los puntos x, y, z
        clashpoint = clashresult.clashpoint.pos3f
        x, y, z = clashpoint['x'], clashpoint['y'], clashpoint['z']

        puntos = [x, y, z]
        
        # Fecha
        anno, mes, dia = clashresult.createddate.date['year'], clashresult.createddate.date['month'], clashresult.createddate.date['day']
        fecha = f'{dia}/{mes}/{anno}'
        
        # Objetos en conflicto
        clashobjects = clashresult.clashobjects
        objetos = []
        
        # Por cada objeto (pueden ser 1 o 2, dependiendo del tipo de conflicto)
        for clashobject in clashobjects.clashobject:
            try:
                # Intentamos obtener el objectattribute
                value = clashobject.objectattribute.value.cdata
            except:
                # Si no lo tiene, adjuntamos un "-" para indicar que no tiene valor
                value = "-"
            layer = clashobject.layer.cdata
            
            smarttags = clashobject.smarttags
            tag_nombre = smarttags.smarttag[0].value.cdata
            tag_tipo = smarttags.smarttag[1].value.cdata
            
            objetos.append([value, tag_nombre, tag_tipo, layer])
        
        # por cada conflicto encontrado en el xml, agregamos una fila al dataframe
        df.loc[i] = [nombre_conflicto, imagen_filename, distancia, result_status, gridlocation, puntos, fecha, objetos]
    
    return nombre_archivo, tolerance, total, activas, resueltas, df


## Dataframe a PDF

In [467]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import pandas as pd

def generar_pdf(dataframe, output_path, folder_path, data_archivo):
    # Configurar el tamaño de la página
    pdf = canvas.Canvas(output_path, pagesize=letter)

    # Iterar sobre las filas del DataFrame
    for _, row in dataframe.iterrows():
        # Agregar una nueva página para cada fila
        

        # Agregar el contenido al PDF para cada columna
        y_pos = 700  # Posición inicial en el eje Y
        ancho_rect = 438 # Ancho de los elementos en el PDF
        padding = 20 # Padding entre los elementos
        
        
        
        # Inicio del documento:
        nombre = data_archivo['Nombre'][:-4]+ " - " + row['Nombre']
        
        #Rectángulo que engloba el titulo de la página
        pdf.rect(100, y_pos-10, ancho_rect, 30, stroke=1, fill=0)
        pdf.setFont("Helvetica-Bold", 16)
        # Centramos el texto en X 
        x_centered = 100 + (400 - pdf.stringWidth(f"{nombre}", "Helvetica-Bold", 16)) / 2
        pdf.drawString(x_centered, y_pos, f"{nombre}")
        
        # Agregar Imagen
        y_pos -= padding
        y_pos -= 200  # Ajustar la posición vertical para la imagen
        
        # Ajustamos el alto del rectángulo
        alto_rect = 200
        pdf.rect(100, y_pos, ancho_rect, alto_rect, stroke=1, fill=0)
        
        
        imagen_path = folder_path+row["Imagen"]  # Suponiendo que 'Imagen' contiene la ruta a la imagen
        
        # La imagen tendrá un 90% del ancho y un 98% del alto del rectángulo
        porcentaje_imagen = 0.9
        ancho_img = ancho_rect*porcentaje_imagen
        alto_img = alto_rect*(porcentaje_imagen+0.08)
        # Centramos la imagen en el rectángulo
        center_x = 100+((ancho_rect-ancho_img)/2)
        center_y = y_pos+((alto_rect-alto_img)/2)
        pdf.drawImage(imagen_path, center_x, center_y, width=ancho_img, height=alto_img)
        
        # Agregar "Informacion del Conflicto"
        y_pos -= padding/2
        y_pos -= 20  # Ajustar la posición vertical para la siguiente línea
        informacion = "Informacion de los Elementos en Conflicto"
        pdf.rect(100, y_pos-10, 438, 30, stroke=1, fill=0)
        pdf.setFont("Helvetica-Bold", 10)
        x_centered = 100 + (400 - pdf.stringWidth(f"{informacion}", "Helvetica-Bold", 10)) / 2
        pdf.drawString(x_centered, y_pos, f"{informacion}")
        
        
        # Tabla de "Solucionar":
        
        y_pos -= padding
        
        # Creamos 4 rectangulos para la tabla
        ancho_celda_4 = 438/4
        y_pos -= 20  # Ajustar la posición vertical para la siguiente línea
        for i in range(4):
            # Por cada rectangulo, calculamos el ancho de la celda y dibujamos el rectangulo
            ancho_celda = 100+(ancho_celda_4*i)
            pdf.rect(ancho_celda, y_pos-10, ancho_celda_4, 30, stroke=1, fill=0)
            
            if i == 0:
                pdf.drawString(ancho_celda+10, y_pos, "Solucionar")
            elif i == 1:
                pdf.drawString(ancho_celda+10, y_pos, f"{row['Estado']}") # Estado
            elif i == 2:
                pdf.drawString(ancho_celda+10, y_pos, f"Fecha")
            elif i == 3:
                pdf.setFont("Helvetica", 10)
                pdf.drawString(ancho_celda+10, y_pos, f"{row['Fecha']}")
        
        y_pos -= padding
        y_pos -= 20  # Ajustar la posición vertical para la siguiente línea
        
        # Misma lógica para "Ubicacion" en X, Y, Z
        for i in range(4):
            ancho_celda = 100+(ancho_celda_4*i)
            pdf.rect(ancho_celda, y_pos-10, ancho_celda_4, 30, stroke=1, fill=0)
            pdf.setFont("Helvetica-Bold", 10) # Texto en negrita
            if i == 0:
                pdf.drawString(ancho_celda+10, y_pos, "Ubicación")
            elif i == 1:
                # Creamos un rectangulo que divida horizontalmente la celda
                pdf.rect(ancho_celda, y_pos-10, ancho_celda_4, 15, stroke=1, fill=0)
                pdf.drawString(ancho_celda+10, y_pos+15/2, f"X") # Grid
                
                punto_to_list = row['Puntos']   # Extraemos los puntos del dataframe
                pdf.setFont("Helvetica", 10)
                pdf.drawString(ancho_celda+10, y_pos-15/2, f"{punto_to_list[0]}") # Punto_X
                
            elif i == 2:
                pdf.rect(ancho_celda, y_pos-10, ancho_celda_4, 15, stroke=1, fill=0)
                pdf.drawString(ancho_celda+10,  y_pos+15/2, f"Y")
                punto_to_list = row['Puntos']
                pdf.setFont("Helvetica", 10)
                pdf.drawString(ancho_celda+10, y_pos-15/2, f"{punto_to_list[1]}")
                
            elif i == 3:
                pdf.rect(ancho_celda, y_pos-10, ancho_celda_4, 15, stroke=1, fill=0)
                pdf.drawString(ancho_celda+10,  y_pos+15/2, f"Z")
                punto_to_list = row['Puntos']
                pdf.setFont("Helvetica", 10)
                pdf.drawString(ancho_celda+10, y_pos-15/2, f"{punto_to_list[2]}")
                
        
        # Tabla para Objetos en Conflicto
        y_pos -= padding
        y_pos -= 20
        pdf.rect(100, y_pos-10, 438, 80, stroke=1, fill=0)
        
        objetos = row['Objetos']    # Extraemos los objetos del dataframe
        
        pdf.setFont("Helvetica-Bold", 10)
        
        
        # A continuacion, dibujamos la tabla de objetos en conflicto con sus respectivos datos
        ancho_celda_3 = 438/3
        for i in range(len(objetos)+1): # Iteramos en la tabla de forma horizontal (3 columnas)
            for j in range(7):  #Iteramos en la tabla de forma vertical
                if i == 0:
                    ancho = 100+(ancho_celda_3*i)
                    ancho_celda_total = ancho_celda_3-30
                    if j == 0:
                        pdf.rect(ancho, y_pos-10, ancho_celda_total, 20, stroke=1, fill=0)
                    elif j == 1:
                        corrimiento = 20
                        pdf.rect(ancho, y_pos-10-corrimiento, ancho_celda_total, 20, stroke=1, fill=0)
                        pdf.drawString(ancho+10, y_pos-5-corrimiento, "Id")
                    elif j == 2:
                        corrimiento = 60
                        pdf.rect(ancho, y_pos-10-corrimiento, ancho_celda_total, 40, stroke=1, fill=0)
                        pdf.drawString(ancho+10, y_pos-corrimiento, "Nombre")
                    elif j == 3:
                        corrimiento = 110
                        pdf.rect(ancho, y_pos-10-corrimiento, ancho_celda_total, 50, stroke=1, fill=0)
                        pdf.drawString(ancho+10, y_pos-5-corrimiento, "Tipo")
                    elif j == 4:
                        corrimiento = 130
                        pdf.rect(ancho, y_pos-10-corrimiento, ancho_celda_total, 20, stroke=1, fill=0)
                        pdf.drawString(ancho+10, y_pos-5-corrimiento, "Nivel")
                    elif j == 5:
                        corrimiento = 150
                        pdf.rect(ancho, y_pos-10-corrimiento, ancho_celda_total, 20, stroke=1, fill=0)
                        pdf.drawString(ancho+10, y_pos-5-corrimiento, "Ejes")
                    elif j == 6:
                        corrimiento = 180
                        pdf.rect(ancho, y_pos-10-corrimiento, ancho_celda_total, 30, stroke=1, fill=0)
                        pdf.drawString(ancho+10, y_pos+5-corrimiento, "Distancia")
                        pdf.drawString(ancho+10, y_pos-5-corrimiento, f"Interferencia")
                    
                elif i == 1:
                    ancho = 100+(ancho_celda_3*i)-30
                    ancho_celda_total = ancho_celda_3+15
                    objeto = objetos[0]
                    ejes = row['Grid']
                    if j == 0:
                        pdf.rect(ancho, y_pos-10, ancho_celda_total, 20, stroke=1, fill=0)
                        pdf.drawString(ancho+10, y_pos-5, "Elemento 1")
                    # ID del elemento
                    elif j == 1:    
                        pdf.setFont("Helvetica", 10)
                        corrimiento = 20
                        pdf.rect(ancho, y_pos-10-corrimiento, ancho_celda_total, 20, stroke=1, fill=0)
                        id_elemento = objeto[0]
                        pdf.drawString(ancho+10, y_pos-5-corrimiento, f"{id_elemento}")
                    
                    # Nombre del objeto
                    elif j == 2:
                        corrimiento = 60
                        pdf.rect(ancho, y_pos-10-corrimiento, ancho_celda_total, 40, stroke=1, fill=0)
                        nombre_elemento = str(objeto[1])
                        ancho_texto = pdf.stringWidth(nombre_elemento, "Helvetica", 10)
                        if ancho_texto > ancho_celda_total:
                            # ajusta el tamaño del texto para que quepa en la celda
                            factor_ajuste = (ancho_celda_total-10)/ancho_texto
                            texto = nombre_elemento[:int(len(nombre_elemento)*factor_ajuste)]
                            pdf.drawCentredString(ancho+80, y_pos+15-corrimiento, f"{texto}")
                            ancho_texto -= ancho_celda_total-10
                            if ancho_texto > ancho_celda_total:
                                nombre_elemento = nombre_elemento[:int(len(nombre_elemento)*factor_ajuste)]
                                pdf.drawCentredString(ancho+80, y_pos+5-corrimiento, f"{nombre_elemento}")
                        else:
                            # Si el texto cabe, imprimimos normalmente
                            pdf.drawCentredString(ancho+80, y_pos+15-corrimiento, f"{nombre_elemento}")
                        
                    
                    # Tipo
                    elif j == 3:
                        corrimiento = 110
                        pdf.rect(ancho, y_pos-10-corrimiento, ancho_celda_total, 50, stroke=1, fill=0)
                        tipo_elemento = str(objeto[2])
                        
                        ancho_texto = pdf.stringWidth(tipo_elemento, "Helvetica", 10)
                        
                        if ancho_texto > ancho_celda_total:
                            # ajusta el tamaño del texto para que quepa en la celda
                            factor_ajuste = (ancho_celda_total-10)/ancho_texto
                            final_1 = int(len(tipo_elemento)*factor_ajuste)
                            
                            texto = tipo_elemento[:final_1]
                            
                            pdf.drawCentredString(ancho+80, y_pos+25-corrimiento, f"{texto}")
                            
                            final_2 = int((len(tipo_elemento)-final_1)*factor_ajuste)
                            texto_2 = tipo_elemento[final_1+1:2*final_1+1]
                            pdf.drawCentredString(ancho+80, y_pos+20-corrimiento-10, f"{texto_2}")
                            
                                
                        else:
                            pdf.drawCentredString(ancho+80, y_pos+25-corrimiento, f"{tipo_elemento}")
                        
                    # Nivel
                    elif j == 4:
                        corrimiento = 130
                        pdf.rect(ancho, y_pos-10-corrimiento, ancho_celda_total, 20, stroke=1, fill=0)
                        nivel_elemento = objeto[3]
                        pdf.drawString(ancho+10, y_pos-5-corrimiento, f"{nivel_elemento}")
                    # Ejes
                    elif j == 5:
                        # Para el caso de Ejes, el rectangulo es de 2x celdas
                        pdf.setFont("Helvetica-Bold", 10)
                        corrimiento = 150
                        pdf.rect(ancho, y_pos-10-corrimiento, (ancho_celda_total)*2, 20, stroke=1, fill=0)
                        pdf.drawString(ancho+10, y_pos-5-corrimiento, f"{ejes}")
                    # Distancia Interferencia
                    elif j == 6:
                        #Idem al anterior, el rectangulo es de 2x celdas
                        corrimiento = 180
                        pdf.rect(ancho, y_pos-10-corrimiento, (ancho_celda_total)*2, 30, stroke=1, fill=0)
                        distancia_interferencia = row['Distancia']
                        pdf.drawString(ancho+10, y_pos-5-corrimiento, f"{distancia_interferencia}")
                    
                    
                elif i == 2:
                    ancho_celda_total = ancho_celda_3+15
                    ancho = 100+(ancho_celda_3*i)-15
                    objeto = objetos[1]
                    if j == 0:
                        pdf.rect(ancho, y_pos-10, ancho_celda_3+15, 20, stroke=1, fill=0)
                        pdf.setFont("Helvetica-Bold", 10)
                        pdf.drawString(ancho+10, y_pos-5, "Elemento 2")
                    # ID del elemento
                    elif j == 1:
                        pdf.setFont("Helvetica", 10)
                        corrimiento = 20
                        pdf.rect(ancho, y_pos-10-corrimiento, ancho_celda_3+15, 20, stroke=1, fill=0)
                        id_elemento = objeto[0]
                        pdf.drawString(ancho+10, y_pos-5-corrimiento, f"{id_elemento}")
                    # Nombre del Objeto
                    elif j == 2:
                        corrimiento = 60
                        pdf.rect(ancho, y_pos-10-corrimiento, ancho_celda_3+15, 40, stroke=1, fill=0)
                        nombre_elemento = str(objeto[1])
                        ancho_texto = pdf.stringWidth(nombre_elemento, "Helvetica", 10)
                        if ancho_texto > ancho_celda_total:
                            # ajusta el tamaño del texto para que quepa en la celda
                            factor_ajuste = (ancho_celda_total-10)/ancho_texto
                            texto = nombre_elemento[:int(len(nombre_elemento)*factor_ajuste)]
                            pdf.drawCentredString(ancho+80, y_pos+15-corrimiento, f"{texto}")
                            ancho_texto -= ancho_celda_total-10
                            if ancho_texto > ancho_celda_total:
                                nombre_elemento = nombre_elemento[:int(len(nombre_elemento)*factor_ajuste)]
                                pdf.drawCentredString(ancho+80, y_pos+5-corrimiento, f"{nombre_elemento}")
                                
                        else:
                            pdf.drawCentredString(ancho+80, y_pos+15-corrimiento, f"{nombre_elemento}")
                    # Tipo
                    elif j == 3:
                        corrimiento = 110
                        pdf.rect(ancho, y_pos-10-corrimiento, ancho_celda_3+15, 50, stroke=1, fill=0)
                        tipo_elemento = str(objeto[2])
                        ancho_texto = pdf.stringWidth(tipo_elemento, "Helvetica", 10)
                        
                        
                        if ancho_texto > ancho_celda_total:
                            # ajusta el tamaño del texto para que quepa en la celda
                            factor_ajuste = (ancho_celda_total-10)/ancho_texto
                            final_1 = int(len(tipo_elemento)*factor_ajuste)
                            
                            texto = tipo_elemento[:final_1]
                            
                            pdf.drawCentredString(ancho+80, y_pos+25-corrimiento, f"{texto}")
                            
                            final_2 = int((len(tipo_elemento)-final_1)*factor_ajuste)
                            texto_2 = tipo_elemento[final_1+1:2*final_1+1]
                            pdf.drawCentredString(ancho+80, y_pos+20-corrimiento-10, f"{texto_2}")
                            
                                
                        else:
                            pdf.drawCentredString(ancho+80, y_pos+25-corrimiento, f"{tipo_elemento}")
                    # Nivel
                    elif j == 4:
                        corrimiento = 130
                        pdf.rect(ancho, y_pos-10-corrimiento, ancho_celda_3+15, 20, stroke=1, fill=0)
                        nivel_elemento = objeto[3]
                        pdf.drawString(ancho+10, y_pos-5-corrimiento, f"{nivel_elemento}")
        
        y_pos -=200
        
        # Notas:
        pdf.rect(100, y_pos-10, 438, 20, stroke=1, fill=0)
        pdf.setFont("Helvetica-Bold", 10)
        pdf.drawString(110, y_pos-5, "Notas:")
        
        # Espacio en blanco para escribir las notas
        y_pos -= 80
        pdf.rect(100, y_pos-10, 438, 80, stroke=1, fill=0)
        
               
                        

                #y_pos -= 20
                    

        pdf.showPage()  # Agregar una nueva página para la siguiente fila
    # Guardar el PDF
    pdf.save()

### Celda de prueba! Para probar con un solo archivo

In [468]:


input_path = "./content/"
input_folder = "01"
input_data = input_path+input_folder+"/"
file_name = "2.- ARQ_PUE VS EST"
image_folder = input_data+file_name+"_files/"
xml_path = input_data + file_name+".xml"

nombre_archivo, tolerance, total, activas, resueltas, archivo_df = get_info_from_xml(xml_path)
data_archivo = {
    'Nombre': file_name,
    'Tolerancia': tolerance,
    'Total': total,
    'Activas': activas,
    'Resueltas': resueltas
}
print("Nombre: ", file_name, "\t Tolerancia: ", tolerance, "\t Total: ", total, "\t Activas: ", activas, "\t Resueltas: ", resueltas)


#display(archivo_df)
# Ruta de salida para el PDF generado

output_folder = './output/'
output_name = nombre_archivo
output_path = output_folder+input_folder+" - "+output_name+".pdf"


generar_pdf(archivo_df, output_path,image_folder, data_archivo)




Nombre:  2.- ARQ_PUE VS EST 	 Tolerancia:  0.001 	 Total:  22 	 Activas:  21 	 Resueltas:  1


## Todas las carpetas


Para cada subcarpeta de "/contents", ejecutamos el codigo en su respectivo XML y construimos el PDF

In [473]:
# Por cada subcarpeta de la carpeta "content"
# Obtener el nombre de la carpeta
# Obtener el nombre del archivo XML
input_path = "./content/"

for folder in os.listdir(input_path):
    if folder[0] != ".":    # Algunos casos (Mac) tiene archivos ocultos que no son carpetas
        folder_path = input_path+folder+"/" # Si efectivamente es una carpeta, obtenemos su ruta
        for file in os.listdir(folder_path):
            if file[-4:] == ".xml": # Si el archivo es un XML, lo procesamos
                xml_path = folder_path+file
                nombre_archivo, tolerance, total, activas, resueltas, archivo_df = get_info_from_xml(xml_path)
                data_archivo = {
                    'Nombre': file,
                    'Tolerancia': tolerance,
                    'Total': total,
                    'Activas': activas,
                    'Resueltas': resueltas
                }
                # Para mostrar en consola los datos obtenidos
                print("Nombre: ", file, "\t Tolerancia: ", tolerance, "\t Total: ", total, "\t Activas: ", activas, "\t Resueltas: ", resueltas)
                
                # Ruta de salida para el PDF generado
                output_folder = './output/'
                output_name = file[:-4]
                output_path = output_folder+folder+" - "+output_name+".pdf"
                img_path = folder_path+file[:-4]+"_files/"
                
                # Procesamos el dataframe y generamos el PDF
                generar_pdf(archivo_df, output_path, img_path, data_archivo)
                print("PDF generado en: ", output_path)
                print("----------------------------------------------------")
                print("----------------------------------------------------")


Nombre:  4.- ARQ_TAB_VEN VS EST.xml 	 Tolerancia:  0.001 	 Total:  42 	 Activas:  42 	 Resueltas:  0
PDF generado en:  ./output/03 - 4.- ARQ_TAB_VEN VS EST.pdf
----------------------------------------------------
----------------------------------------------------
Nombre:  5.- ARQ_CIELOS VS EST.xml 	 Tolerancia:  0.001 	 Total:  28 	 Activas:  28 	 Resueltas:  0
PDF generado en:  ./output/04 - 5.- ARQ_CIELOS VS EST.pdf
----------------------------------------------------
----------------------------------------------------
Nombre:  ARQ_VEN VS EST.xml 	 Tolerancia:  0.001 	 Total:  120 	 Activas:  0 	 Resueltas:  0
PDF generado en:  ./output/32 - ARQ_VEN VS EST.pdf
----------------------------------------------------
----------------------------------------------------
Nombre:  ARQ_PUE VS EST.xml 	 Tolerancia:  0.001 	 Total:  22 	 Activas:  0 	 Resueltas:  0
PDF generado en:  ./output/35 - ARQ_PUE VS EST.pdf
----------------------------------------------------
------------------------